# 20th Century Book and Their Movie Adaptations
Exploration of the popularity of titles around the time of their publication and movie adaptation.

By Kenneth Yang

# Introduction
## What are we looking for?
- 📖 How did a title's popularity trend when it was published?
- 🎥 How did a title's popularity trend when it was adapted into a movie?
- 📈 Did the movie adaptation leave a net increase in popularity of the title?

## What is Google Ngram Viewer and why are we using it?
- 🔎 [Google Books Ngram Viewer](https://books.google.com/ngrams/)
- 📰 20th century: most talk is through books, newspapers, and magazines. Works as a proxy for popularity.
- 📊 Makes pretty graphs for us to look at.

# Data Collection
- Define books, their publication dates, and their movie adaptations.
- Pull the data from Google Ngram Viewer.

## Publication dates of books and their movie adaptations

In [ ]:
from collections import namedtuple

from debugpy.common.messaging import Response

Published = namedtuple("Published", ["book", "movie"])

In [ ]:
books = {
    "Gone with the Wind": Published(1936, 1939),
    "The Grapes of Wrath": Published(1939, 1940),
    "The Maltese Falcon": Published(1930, 1941),
    "To Kill a Mockingbird": Published(1960, 1962),
    "One Flew Over the Cuckoo's Nest": Published(1962, 1975),
    "The Godfather": Published(1969, 1972),
    "A Clockwork Orange": Published(1962, 1971),
}

## Pull data from Google Ngram Viewer

In [ ]:
from requests import get
def json_url(title: str, start=1900, end=2000):
    return f"https://books.google.com/ngrams/json?content={title.replace(" ", "+")}&year_start={start}&year_end={end}&corpus=en&smoothing=0&case_insensitive=true"

In [ ]:
responses = {}
for book in books:
    response = get(json_url(book))
    response.raise_for_status()
    responses[book] = response.json()[0]["timeseries"]

# Display the Data

In [ ]:
from IPython.display import IFrame


def graph_url(title: str, start=1900, end=2000):
    return json_url(title, start, end).replace("json", "interactive_chart", 1)

In [ ]:
IFrame(graph_url("Gone with the Wind"), width=900, height=500) 